# Control Barrier Functions

This notebook introduces the concept of control barriers and illustrates how to use them in Pink.

In [1]:
import meshcat_shapes
import numpy as np
import qpsolvers
from loop_rate_limiters import RateLimiter

import pink
from pink import solve_ik
from pink.barriers import PositionBarrier, ConfigurationBarrier, BodySphericalBarrier
from pink.tasks import FrameTask, PostureTask
from pink.visualization import start_meshcat_visualizer
import pinocchio as pin

from IPython.display import display

from robot_descriptions.loaders.pinocchio import load_robot_description

In [2]:
# Robot model and basic configuration for UR5 robot
ur5_robot = load_robot_description("ur5_description", root_joint=None)
ur5_q_ref = np.array(
    [
        1.27153374,
        -0.87988708,
        1.89104795,
        1.73996951,
        -0.24610945,
        -0.74979019,
    ]
)
ur5_configuration = pink.Configuration(ur5_robot.model, ur5_robot.data, ur5_q_ref)

In [3]:
# Robot model and basic configuration for Yumi robot
yumi_robot = load_robot_description("yumi_description", root_joint=None)

# Introducing new frames, which will be used for collision-avoidance barrier later:
# Left frame
l_frame_placement = pin.SE3()
l_frame_placement.translation = np.array([0.0, 0.0, 0.05])
l_frame_placement.rotation = np.eye(3)

l_frame = pin.Frame(
    "yumi_barrier_l",
    yumi_robot.model.getJointId("yumi_joint_6_l"),
    yumi_robot.model.getFrameId("yumi_link_7_l"),
    l_frame_placement,
    pin.FrameType.OP_FRAME,
)

# Right frame
r_frame_placement = pin.SE3()
r_frame_placement.translation = np.array([0.0, 0.0, 0.05])
r_frame_placement.rotation = np.eye(3)

r_frame = pin.Frame(
    "yumi_barrier_r",
    yumi_robot.model.getJointId("yumi_joint_6_r"),
    yumi_robot.model.getFrameId("yumi_link_7_r"),
    r_frame_placement,
    pin.FrameType.OP_FRAME,
)

yumi_robot.model.addFrame(l_frame)
yumi_robot.model.addFrame(r_frame)

yumi_robot.data = pin.Data(yumi_robot.model)


yumi_q_ref = np.array(
    [
        0.045,
        -0.155,
        -0.394,
        -0.617,
        -0.939,
        -0.343,
        -1.216,
        0,
        0,
        -0.374,
        -0.249,
        0.562,
        -0.520,
        0.934,
        -0.337,
        1.400,
        0,
        0,
    ]
)
yumi_configuration = pink.Configuration(yumi_robot.model, yumi_robot.data, yumi_q_ref)

# Introduction

Control barrier functions (CBFs) are used to enforce safety for nonlinear control affine systems of the form:

$$
\dot{x}=f(x)+g(x) u
$$

where $f$ and $g$ are locally Lipschitz, $x \in D \subset \mathbb{R}^{n}$, and $u \in U \subset \mathbb{R}^{m}$ is the set of admissible inputs.

## Safety and Safe Sets


Safety is framed in the context of enforcing invariance of a set, i.e., not leaving a safe set. Consider a set $\mathcal{S}$ defined as the superlevel set of a continuously differentiable function $h: D \subset \mathbb{R}^{n} \rightarrow \mathbb{R}$:

$$
\begin{aligned}
\mathcal{S} & =\left\{x \in D \subset \mathbb{R}^{n}: h(x) \geq 0\right\} \\
\partial \mathcal{S} & =\left\{x \in D \subset \mathbb{R}^{n}: h(x)=0\right\} \\
\operatorname{Int}(\mathcal{S}) & =\left\{x \in D \subset \mathbb{R}^{n}: h(x)>0\right\}
\end{aligned}
$$

We refer to $\mathcal{S}$ as the safe set. The system is safe with respect to the set $\mathcal{S}$ if the set $\mathcal{S}$ is forward invariant. Formally, the set $\mathcal{S}$ is forward invariant if for every $x_{0} \in \mathcal{S}$, the solution $x(t)$ satisfying $x(0)=x_0$ remains in $\mathcal{S}$ for all $t$.

## Control Barrier Functions


A function $h$ is a control barrier function (CBF) if there exists an extended class $\mathcal{K}_{\infty}$ function $\alpha$ such that:

$$
\sup _{u \in U}\left[L_{f} h(x)+L_{g} h(x) u\right] \geq-\alpha(h(x))
$$

for all $x \in D$. Here $L_f h$ and $L_g h$ denote the Lie derivatives of $h$ along $f$ and $g$.

The set of all control values that render $\mathcal{S}$ safe is:

$$
K_{\mathrm{cbf}}(x)=\left\{u \in U: L_{f} h(x)+L_{g} h(x) u+\alpha(h(x)) \geq 0\right\}
$$

The main result regarding CBFs is that their existence implies the control system is safe:

**Theorem:** Let $\mathcal{S} \subset \mathbb{R}^{n}$ be a set defined as the superlevel set of a continuously differentiable function $h: D \subset \mathbb{R}^{n} \rightarrow \mathbb{R}$. If $h$ is a CBF on $D$ and $\frac{\partial h}{\partial x}(x) \neq 0$ for all $x \in \partial \mathcal{S}$, then any Lipschitz continuous controller $u(x) \in K_{\mathrm{cbf}}(x)$ renders the set $\mathcal{S}$ safe and asymptotically stable in $D$.

## Optimization Based Control

To synthesize safety-critical controllers, we can solve a quadratic program (QP) to minimally modify an existing controller $k(x)$ to guarantee safety:

$$
\begin{aligned}
u(x)=\underset{u \in \mathbb{R}^{m}}{\operatorname{argmin}} & \frac{1}{2}\|u-k(x)\|^{2} \quad(\mathrm{CBF}-\mathrm{QP}) \\
\text { s.t. } & L_{f} h(x)+L_{g} h(x) u \geq-\alpha(h(x))
\end{aligned}
$$

When there are no input constraints ($U=\mathbb{R}^m$), the CBF-QP has a closed-form solution given by the min-norm controller.

## Discrete-Time Implementations

Discrete-time implementations of continuous-time CBF-based safety filters can lead to some challenges. When solving the CBF-QP at a sampling time $\Delta t>0$, safety is only guaranteed at the initial time step $t_{0}$, but not for the open time interval $(t_{0}, t_{0}+\Delta t)$. Suboptimal performance can arise when $\|L_{g} L_{f}^{s-1} h(x)\| \rightarrow 0$, as large inputs permissible by the CBF condition applied over the finite time interval could result in undesirable behavior.

Another issue occurs when $L_{g} L_{f}^{s-1} h(x)=0$, indicating a local relative degree higher than $s$. In such cases, the safety controller becomes inactive, allowing potentially unsafe control inputs $k(x)$ to be applied for at least the time interval $[t_{0}, t_{0}+\Delta t)$, which may lead to safe set violations or suboptimal performance.

One method to handle the case of $\|L_{g} L_{f}^{s-1} h(x)\| \rightarrow 0$ is by modifying the safety filter objective function. We can add a term that explicitly accounts for $L_{g} L_{f}^{s-1} h(x)$ becoming close to 0. The proposed modified safety filtering objective is:

$$
J(x)=\frac{1}{2}\|u-k(x)\|^{2}+\frac{r}{2\|L_{g} L_{f}^{s-1} h(x)\|^{2}}\|u-k_{\text{safe}}(x)\|^{2}
$$

where $k_{\text{safe}}$ is a known safe backup control policy (e.g., a stabilizing controller that renders $\mathcal{S}$ control invariant). The parameter $r>0$ is a weighting factor.

This new objective replaces the standard CBF-QP objective for all $\|L_{g} L_{f}^{s-1} h(x)\|>\epsilon$, where $\epsilon$ is a small positive number. The closer $\|L_{g} L_{f}^{s-1} h(x)\|$ gets to 0, the greater the impact of the second term in the safety filtering objective. In this case, the safety filter will track the safe backup control policy instead of the potentially unsafe control policy $k(x)$. The weighting parameter $r$ determines the balance between the two terms when $\|L_{g} L_{f}^{s-1} h(x)\|$ is far from 0. To avoid numerical instabilities, we set $u(x)=k_{\text{safe}}(x)$ when the system is in a state $x$ such that $\|L_{g} L_{f}^{s-1} h(x)\| \leq \epsilon$.

The modified CBF-QP with the penalty term becomes:

$$
\begin{aligned}
u(x)=\underset{u \in \mathbb{R}^{m}}{\operatorname{argmin}} & \frac{1}{2}\|u-k(x)\|^{2}+\frac{r}{2\|L_{g} L_{f}^{s-1} h(x)\|^{2}}\|u-k_{\text{safe}}(x)\|^{2} \\
\text{s.t.} & L_{f} h(x)+L_{g} h(x) u \geq-\alpha(h(x))
\end{aligned}
$$

This strategy requires almost no additional computational effort. However, in practice, the design of the safe backup control policy $k_{\text{safe}}$ will require some attention. The backup policy should be able to return the system to states where $\|L_{g} L_{f}^{s-1} h(x)\|>\epsilon$. Otherwise, the system will continue using the backup control policy $k_{\text{safe}}$ for all future time.

## CBF in Differential IK


Let us study how CBF theory can be applied to solve the multitask inverse kinematics problem subject to configuration-based nonlinear inequalities. 

Mathematically, our goal is to find the configuration motion $\boldsymbol{q}(t) \in \mathcal{C}$ such that:

$$
\boldsymbol{r}_i(\boldsymbol{q})=\boldsymbol{p}_i^{*}-\boldsymbol{p}_i(\boldsymbol{q})
$$

where $\boldsymbol{p}_i^{*}$ is the desired task reference, $\boldsymbol{r}_i(\boldsymbol{q})$ is the task residual error, and $\boldsymbol{p}_i(\boldsymbol{q})$ is the forward kinematics function mapping configuration variables to task space.

In addition to the kinematic tasks, suppose we want to enforce the following nonlinear safety constraints:

$$
h_j(\boldsymbol{q}) \geq 0
$$

Directly solving this constrained optimization problem can lead to challenging nonlinear programs. However, in the context of differential inverse kinematics, we can leverage CBFs to reformulate the problem as a quadratic program (QP) with linear inequality constraints.


## CBF QP Formulation

Differentiating the kinematic task errors once and introducing stabilizing feedback gains yields the following QP formulation:

$$
\underset{\dot{\boldsymbol{q}}}{\operatorname{minimize}} \sum_{\operatorname{task} i} w_{i}\left\|\boldsymbol{J}_{i} \dot{\boldsymbol{q}}-K_{i} \boldsymbol{v}_{i}\right\|^{2}
$$

Here, $\boldsymbol{J}_i$ is the task Jacobian, $K_i$ is a positive definite feedback gain matrix, and $\boldsymbol{v}_i$ is an auxiliary control input. By treating joint velocities $\dot{\boldsymbol{q}}$ as optimization variables, we can incorporate the safety constraints $h_j(\boldsymbol{q}) \geq 0$ as CBF conditions:

$$
\dot{h}_j(\boldsymbol{q})+\alpha_j(h_j(\boldsymbol{q})) = \frac{\partial h_j}{\partial \boldsymbol{q}} \dot{\boldsymbol{q}} +\alpha_j(h_j(\boldsymbol{q})) \geq 0
$$

Enforcing this constraint ensures the system remains within the safe set $\mathcal{S}_j =\left\{\boldsymbol{q} \in D \subset \mathbb{R}^{n}: h_j(\boldsymbol{q}) \geq 0\right\}$. The extended class $\mathcal{K}$ function $\alpha_j$ provides a safety margin.

Combining the kinematic tasks and CBF constraints, we arrive at the differentiable IK optimization problem with safety guarantees:

$$
\begin{aligned}
& \underset{\dot{\boldsymbol{q}}}{\operatorname{minimize}} \sum_{\text {task } i} w_{i}\left\|\boldsymbol{J}_{i}(\boldsymbol{q}) \dot{\boldsymbol{q}}-K_{i} \boldsymbol{v}_{i}\right\|^{2} + \gamma(\boldsymbol{q})\left\| \dot{\boldsymbol{q}}-\dot{\boldsymbol{q}}_{safe}(\boldsymbol{q})\right\|^{2} \\
& \text { subject to: } \frac{\partial \boldsymbol{h}_j}{\partial \boldsymbol{q}} \dot{\boldsymbol{q}} +\alpha_j(\boldsymbol{h}_j(\boldsymbol{q})) \geq 0, \quad \forall j
\end{aligned}
$$

The configuration-dependent weight $\gamma(\boldsymbol{q})$, often chosen as $(\|\partial h_j \|  +\epsilon)^{-1}w_h$, ensures safety takes precedence when close to constraint boundaries. The safe backup policy $\dot{\boldsymbol{q}}_{safe}(\boldsymbol{q})$ provides a fallback when $\|\partial h_j \|$ approaches zero. A simple choice is the zero-velocity policy $\dot{\boldsymbol{q}}_{safe} = \boldsymbol{0}$, which stops the robot. Alternatively, one can stabilize to a safe initial configuration.

# Control Barrier Functions in Pink

In pink, Contorl Barrier Functions are implemented based on `Barrier` class. It defines general logic of computing the cost and constraint, using barrier function $\boldsymbol{h}$ and its jacobian $\frac{\partial h_j}{\partial \boldsymbol{q}}$. Also, each specific barrier might overwrite a safe backup policy $\dot{\boldsymbol{q}}_{safe}(\boldsymbol{q})$, which equals to zero by default.

The following sections describe the implementation of each barrier in pink.

## Position barrier

Position barrier incorporates constraint on the position of specified frame:
$$p_{min} \leq p \leq p_{max}$$
which could be writeen in the form $h(q) \geq 0$ as:
$$h_j(\boldsymbol{q}) = \begin{cases} 
      p_j - p_j^{min} & j = 1,\dots,n \\
      p_j^{max} - p_j & j = n+1,\dots,2n
   \end{cases}$$


which is incorporated as single constraint with optional minimal and maximal bounds. Also, it could be applied not to all positions of the frame, but only to specified indices of the position.

See the following example of position barrier, applied to the end-effector of UR5 robot:

In [4]:
# ==== Model =====
# Update initial configuration
ur5_configuration.update(ur5_q_ref)

# ===== Tasks =====
# Task for tracking end-effector
end_effector_task = FrameTask(
    "ee_link",
    position_cost=50.0,  # [cost] / [m]
    orientation_cost=1.0,  # [cost] / [rad]
)
posture_task = PostureTask(
    cost=1e-3,  # [cost] / [rad]
)
tasks = [end_effector_task, posture_task]

# Initializing task target values
for task in tasks:
    task.set_target_from_configuration(ur5_configuration)

# ===== Barriers =====
# Barrier for end-effector, which limits maximal y coordinate 
# value to 0.6
pos_cbf = PositionBarrier(
    "ee_link",
    indices=[1],    # Select only y coordinate
    max=np.array([0.6]),
    gain=np.array([100.0]),
    r=1.0,
)
cbf_list = [pos_cbf]

# ===== Visualization =====
viz = start_meshcat_visualizer(ur5_robot, open=False)
viewer = viz.viewer
display(viewer.jupyter_cell())

viz.display(ur5_configuration.q)

# Initializing visualization frames
meshcat_shapes.frame(viewer["end_effector_target"], opacity=1.0)
meshcat_shapes.frame(viewer["end_effector"], opacity=1.0)

# ===== Solver =====
# Select QP solver
solver = qpsolvers.available_solvers[0]
if "ecos" in qpsolvers.available_solvers:
    solver = "ecos"

# ===== Control Loop =====
rate = RateLimiter(frequency=150.0)
dt = rate.period
t = 0.0  # [s]
while t < 20:
    # Update task targets
    end_effector_target = end_effector_task.transform_target_to_world
    end_effector_target.translation[1] = 0.0 + 0.7 * np.sin(t / 2)
    end_effector_target.translation[2] = 0.2

    # Update visualization frames
    viewer["end_effector_target"].set_transform(end_effector_target.np)
    viewer["end_effector"].set_transform(ur5_configuration.get_transform_frame_to_world(end_effector_task.frame).np)

    # Compute velocity and integrate it into next configuration
    # Note that default position limit handle given trajectory
    # much worse than CBF. Hence, we disable it here.
    velocity = solve_ik(
        ur5_configuration,
        tasks,
        dt,
        solver=solver,
        cbfs=cbf_list,
    )
    ur5_configuration.integrate_inplace(velocity, dt)

    # Visualize result at fixed FPS
    viz.display(ur5_configuration.q)
    rate.sleep()
    t += dt


You can open the visualizer by visiting the following URL:
http://127.0.0.1:7022/static/


As you could see, the position barrier prevented the robot from going out of the safe set, i.e. $y_{ee} \leq 0.6$. 

Apart from limiting the position of the frame, it also allows to implement joint limits.

## Joint Barrier

Joint barrier incorporates constraint on the joints given by urdf model:
$$q_{min} \leq q \leq q_{max}$$
which could be writeen in the form $h(q) \geq 0$ as:
$$h_j(\boldsymbol{q}) = \begin{cases} 
      q_j - q_j^{min} & j = 1,\dots,n \\
      q_j^{max} - q_j & j = n+1,\dots,2n
   \end{cases}$$



Let's add joint barrier to the previous example:

In [5]:
# ==== Model =====
# Update initial configuration
ur5_configuration.update(ur5_q_ref)

# ===== Tasks =====
# Initializing task target values
for task in tasks:
    task.set_target_from_configuration(ur5_configuration)

# ===== Barriers =====
# Barrier for end-effector, which limits maximal y coordinate 
# value to 0.6

configuration_cbf = ConfigurationBarrier(ur5_robot.model, gain=100, r=1.0)
cbf_list = [pos_cbf, configuration_cbf]

# ===== Visualization =====
viz = start_meshcat_visualizer(ur5_robot, open=False)
viewer = viz.viewer
display(viewer.jupyter_cell())

viz.display(ur5_configuration.q)

# Initializing visualization frames
meshcat_shapes.frame(viewer["end_effector_target"], opacity=1.0)
meshcat_shapes.frame(viewer["end_effector"], opacity=1.0)

# ===== Solver =====
# Select QP solver
solver = qpsolvers.available_solvers[0]
if "ecos" in qpsolvers.available_solvers:
    solver = "ecos"

# ===== Control Loop =====
rate = RateLimiter(frequency=150.0)
dt = rate.period
t = 0.0  # [s]
while t < 20:
    # Update task targets
    end_effector_target = end_effector_task.transform_target_to_world
    end_effector_target.translation[1] = 0.0 + 0.7 * np.sin(t / 2)
    end_effector_target.translation[2] = 0.2

    # Update visualization frames
    viewer["end_effector_target"].set_transform(end_effector_target.np)
    viewer["end_effector"].set_transform(ur5_configuration.get_transform_frame_to_world(end_effector_task.frame).np)

    # Compute velocity and integrate it into next configuration
    # Note that default position limit handle given trajectory
    # much worse than CBF. Hence, we disable it here.
    velocity = solve_ik(
        ur5_configuration,
        tasks,
        dt,
        solver=solver,
        cbfs=cbf_list,
        use_position_limit=False,  # Disable joint limit handle
    )
    ur5_configuration.integrate_inplace(velocity, dt)

    # Visualize result at fixed FPS
    viz.display(ur5_configuration.q)
    rate.sleep()
    t += dt


You can open the visualizer by visiting the following URL:
http://127.0.0.1:7023/static/


The results are the same as in previous example, but the paradigm has changed.

In the simulation, one could notice that some self-collisions are present. This is obviously not acceptable in the real world. Therefore, the next section describes the implementation of collision-avoidance barriers.

## Collision-Avoidance Barrier

The collision avoidance barrier is formulated as minimal distance between two specified frames:
$$||(p_i(\boldsymbol{q}), p_k(\boldsymbol{q}))||_2^2 \geq d_{safe}^2$$
which could be writeen in the form $h(q) \geq 0$ as:
$$h_j(\boldsymbol{q}) = ||(p_i(\boldsymbol{q}), p_k(\boldsymbol{q}))||_2^2 - d_{safe}^2$$

Let's examine how it works using example with two-armed manipulator `yumi`:

In [7]:
# ==== Tasks =====
# end-effector tracking tasks
left_ee_task = FrameTask(
    "yumi_link_7_l",
    position_cost=50.0,  # [cost] / [m]
    orientation_cost=1.0,  # [cost] / [rad]
)
right_ee_task = FrameTask(
    "yumi_link_7_r",
    position_cost=50.0,  # [cost] / [m]
    orientation_cost=1.0,  # [cost] / [rad]
)
# Posture task for regulation
posture_task = PostureTask(
    cost=1e-3,  # [cost] / [rad]
)
tasks = [left_ee_task, right_ee_task, posture_task]

for task in tasks:
    task.set_target_from_configuration(yumi_configuration)

# ===== Barriers =====
# End-effector collision-avoidance barrier
ee_barrier = BodySphericalBarrier(
    ("yumi_barrier_l", "yumi_barrier_r"),
    d_min=0.2,
    gain=100.0,
    r=1.0,
)
# Joint limit barrier
configuration_cbf = ConfigurationBarrier(yumi_robot.model, gain=1, r=400.0)
cbf_list = [ee_barrier, configuration_cbf]

# ==== Visualization =====
viz = start_meshcat_visualizer(yumi_robot, open=False)
viewer = viz.viewer
display(viewer.jupyter_cell())

viz.display(yumi_configuration.q)

meshcat_shapes.frame(viewer["left_end_effector_target"], opacity=1.0)
meshcat_shapes.frame(viewer["right_end_effector_target"], opacity=1.0)
meshcat_shapes.sphere(
    viewer["left_ee_barrier"],
    opacity=0.4,
    color=0xFF0000,
    radius=0.1,
)
meshcat_shapes.sphere(
    viewer["right_ee_barrier"],
    opacity=0.4,
    color=0x00FF00,
    radius=0.1,
)

# ==== Solver =====
# Select QP solver
solver = qpsolvers.available_solvers[0]
if "ecos" in qpsolvers.available_solvers:
    solver = "ecos"

# ===== Control Loop =====
rate = RateLimiter(frequency=100.0)
dt = rate.period
t = 0.0  # [s]

# Desired position values
l_y_des = np.array([0.392, 0.392, 0.6])
r_y_des = np.array([0.392, 0.392, 0.6])
l_dy_des = np.zeros(3)
r_dy_des = np.zeros(3)

while t < 20:
    # Desired trajectories is sinusoidal waves in y and z directions.
    # They also tend to collide with each other.
    l_y_des[:] = 0.6, 0.1 + 0.25 * np.sin(2 * t), 0.6 + 0.1 * np.sin(2 * t)
    r_y_des[:] = 0.6, -0.1 - 0.25 * np.sin(2 * t), 0.6 + 0.1 * np.sin(2 * t)

    left_ee_task.transform_target_to_world.translation = l_y_des
    right_ee_task.transform_target_to_world.translation = r_y_des

    # Update visualization frames
    viewer["left_end_effector_target"].set_transform(left_ee_task.transform_target_to_world.np)
    viewer["right_end_effector_target"].set_transform(right_ee_task.transform_target_to_world.np)
    viewer["left_ee_barrier"].set_transform(yumi_configuration.get_transform_frame_to_world("yumi_barrier_l").np)
    viewer["right_ee_barrier"].set_transform(yumi_configuration.get_transform_frame_to_world("yumi_barrier_r").np)

    # Compute velocity and integrate it into next configuration
    # Note that default position limit handle given trajectory
    # much worse than CBF. Hence, we disable it here.
    velocity = solve_ik(
        yumi_configuration,
        tasks,
        dt,
        solver=solver,
        cbfs=cbf_list,
        use_position_limit=False,
    )
    yumi_configuration.integrate_inplace(velocity, dt)
    # Visualize result at fixed FPS
    viz.display(yumi_configuration.q)
    rate.sleep()
    t += dt


You can open the visualizer by visiting the following URL:
http://127.0.0.1:7025/static/


In the example, specified frames did not collide, despite task values motivated them to do so.